# Build HTML Doc from Plots

In [ ]:
!pip install git+https://github.com/ZachWolpe/forexflaggr.git

In [ ]:
import forexflaggr as fxr

from pygam import LinearGAM, s, f, te
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import itertools
import warnings
warnings.filterwarnings('ignore')

# Parameters ----------------------------------------------------------->>
_n_days     = 500
_MA_periods = int(12*14) # 2 weeks (1 sample/hour * 14 days)
# Parameters ----------------------------------------------------------->>

# USDZAR
ff = fxr.ForexFlaggr()
ff\
    .fetch_data(stock='USDZAR=X', n_days=_n_days)\
    .plot_signal(MA_periods=_MA_periods)
ff.fig.show()

# USTreasury Bills
fi = fxr.ForexFlaggr()
fi\
    .fetch_data(stock='^IRX', n_days=_n_days)\
    .plot_signal(MA_periods=_MA_periods)
fi.fig.show()

# S&P500
fs = fxr.ForexFlaggr()
fs\
    .fetch_data(stock='^GSPC', n_days=_n_days)\
    .plot_signal(MA_periods=_MA_periods)
fs.fig.show()


# save plots as html
ff.fig.write_html("output/plots/usdzar.html")
fi.fig.write_html("output/plots/UStreasury.html")
fs.fig.write_html("output/plots/sp500.html")

# Price data
_datetime, _timezone, _close, _open, _high, _low = fxr.get_price(ff)


# Price momentum
_n_samples = 252
pcr_fig = fxr.pie_chart_recommendation.plot_pie_recommendation(ff.df_all, n_samples=_n_samples, fig=go.Figure())
pcr_fig.write_html("output/plots/pie_chart_recommendation.html")
pcr_fig.show()

# GAMs + LOESS
# extract data
model_data  = ff.data.reset_index()
X, y        = np.array(model_data.index), np.array(model_data.Close)

# build LOESS model
loess_model = fxr.LOESS_Model(X, y)
loess_model\
    .build()\
    .plot_prediction()\
    .fig.show()
loess_model.fig.write_html("output/plots/loess.html")

# build GAM model
model_data  = ff.data.reset_index()
X,y         = list(model_data.index), model_data.Close
X,y         = np.reshape(X, (-1, 1)), np.reshape(y.values, (-1, 1))


# build GAM model
gam = fxr.GAM_Model(X, y)
gam.build()
gam.fig.write_html("output/plots/gam.html")

# Complex Model
# 1. build dataframe
def transform_df(ff, col_name='USDZAR'):
    df = ff.df_all.copy()
    df.index = df.index.strftime('%Y-%m-%d').copy()
    df = df.groupby(df.index).mean()
    df = df[['Close']]
    df.columns = [col_name]
    return df

df = transform_df(ff, 'USDZAR').join(transform_df(fi, 'USTBills')).join(transform_df(fs, 'S&P500')).reset_index()


# 2. plot 3d scatter plot
fig = px.scatter_3d(df, x='Datetime', y='USTBills', z='USDZAR', color='S&P500')
fig.update_layout(template='plotly_dark', title='USDZAR ~ (USTreasury, S&P500)')
fig.write_html("output/plots/3d_scatter.html")

# 3. fit gam: note use index in place of date as a numeric
df = df.dropna()
gam = LinearGAM(s(0) + s(1) + s(2)).fit(df.reset_index()[['index', 'USTBills', 'S&P500']], df['USDZAR'])
gam.summary()

# 4. plot gam
fig, axs = plt.subplots(1,3)
titles = ['USDZAR', 'USTBills', 'S&P500']
for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    pdep, confi = gam.partial_dependence(term=i, width=.95)
    ax.plot(XX[:, i], pdep)
    ax.plot(XX[:, i], confi, c='r', ls='--')
    ax.set_title(titles[i])

# save
try:
    fig.savefig('output/plots/gam.png')
except Exception:
    pass

# 5. Make Prediction
def gam_prediction(gam=gam, df=df):
    df          = df.copy()
    yhat        = gam.predict(df.reset_index()[['index', 'USTBills', 'S&P500']])
    df['yhat']  = yhat
    return df

df2 = gam_prediction()
df2.head()

# 6. Plot prediction vs actual: create df
df_plot             = df.copy()
df_plot['source']   = 'actual'
df_plot2            = df2.copy()
df_plot2['source']  = 'predicted'
df_plot2['USDZAR']  = df_plot2['yhat']
df_plot2            = df_plot2.drop(columns=['yhat']) 
df_plot             = pd.concat([df_plot, df_plot2], axis=0)

# 6. Plot prediction vs actual: build plot
fig = px.scatter_3d(df_plot, x='Datetime', y='USTBills', z='USDZAR', color='S&P500', symbol='source', color_continuous_scale='turbo')
fig.update_layout(template=None, title='USDZAR ~ (USTreasury, S&P500)')
fig.write_html("output/plots/3d_scatter_prediction.html")

# 7. Plot over a larger prediction plane to examine the model: build prediction space
Z = pd.DataFrame(list(itertools.product(
    df.reset_index()['index'],
    df['USTBills'].unique(),
    df['S&P500'].unique()
    )), columns=['index', 'USTBills', 'S&P500'])


# 7. Plot over a larger prediction plane to examine the model: 
#   take every 100 sample, to downsample the data
_Z = Z.iloc[::100, :]

# fit
yhat        = gam.predict(_Z)
_Z          = _Z.set_index('index').join(df['Datetime'])
_Z['yhat']  = yhat


# transform to grid
# _Z.set_index('Datetime', inplace=True)
_Z = _Z.pivot_table(index='Datetime', columns='USTBills', values='yhat')

# 8. plot
fig = go.Figure(data=[go.Surface(z=_Z.values, x=_Z.index, y=_Z.columns)])
fig = go.Figure(data=[
    go.Surface(z=_Z.values, x=_Z.index, y=_Z.columns),
    go.Scatter3d(x=df['Datetime'], y=df['USTBills'], z=df['USDZAR'], mode='markers',
    marker=dict(size=5, color='lightblue',line=dict(color='darkblue', width=1.5)))
    ])

# change legend header
fig.update_layout(scene = dict(
                    xaxis_title='',
                    yaxis_title='USTBills Yield',
                    zaxis_title='USDZAR',
    ),
    legend=dict(title='S&P500', yanchor="top", y=0.99, xanchor="left", x=0.01, font=dict(size=1200, color="white"))

                    )
fig.update_layout(title='USDZAR ~ (Datetime, US-TBills, S&P500)', template='plotly_dark')
fig.show()

fig.write_html("output/plots/3d_hyperplane_scatter.html")
